In [ ]:
import pylab

import numpy as np
from scipy.io import wavfile

from scipy.signal import correlate
from scipy.signal import correlation_lags

In [ ]:
# NOTE(dominic): This is super lame, but iptyhon kernels are running in the directory they are
# contained. That means, we cannot access any other sub-modules from within these notebooks.
# We resolve this by temporarily adding the parent directory to the path.

import os
import sys
sys.path.insert(0, os.path.abspath('..'))


from util.data_io import read_sdriq_data
from util.demodulation import chunked_demodulate_signal
from util.time_sync import compute_time_lag_in_samples, shift_signals_with_lag

In [ ]:
signal = read_sdriq_data('/Users/dominic/Software/SDR_satellite_tracking/data/dom_test_6.sdriq')

fs = 2e6

dom_signal_crops = list()
band_filter_cutoffs = list()

dom_signal_crops.append( signal[:int(8 * fs)] )
band_filter_cutoffs.append(80e3)
dom_signal_crops.append( signal[int(12 * fs):int(18 * fs)] )
band_filter_cutoffs.append(15e3)
dom_signal_crops.append( signal[int(22 * fs):] )
band_filter_cutoffs.append(80e3)

signal = None

dom_output_audio_signals = list()


for i, crop_signal in enumerate(dom_signal_crops):
    band_filter_cutoff = band_filter_cutoffs[i]

    audio_signal, audio_signal_fs = chunked_demodulate_signal(crop_signal, sample_rate=fs,
                                                             band_filter_cutoff=band_filter_cutoff)
    wavfile.write(f'/Users/dominic/Software/SDR_satellite_tracking/data/dom_test_6_crop{i}.wav',
                  int(audio_signal_fs), audio_signal)
        
    dom_output_audio_signals.append(audio_signal)

In [ ]:
signal = read_sdriq_data('/Users/dominic/Software/SDR_satellite_tracking/data/ben_test_6.sdriq')

fs = 2e6

ben_signal_crops = list()

ben_signal_crops.append( signal[:int(8 * fs)] )
ben_signal_crops.append( signal[int(12 * fs):int(18 * fs)] )
ben_signal_crops.append( signal[int(22 * fs):] )

signal = None


ben_output_audio_signals = list()


for i, crop_signal in enumerate(ben_signal_crops):

    band_filter_cutoff = band_filter_cutoffs[i]

    audio_signal, audio_signal_fs = chunked_demodulate_signal(crop_signal, sample_rate=fs,
                                                             band_filter_cutoff=band_filter_cutoff)
    wavfile.write(f'/Users/dominic/Software/SDR_satellite_tracking/data/ben_test_6_crop{i}.wav',
                  int(audio_signal_fs), audio_signal)
    
    ben_output_audio_signals.append(audio_signal)

In [ ]:
unsynced_combined_audio_crop0 = dom_output_audio_signals[0] + ben_output_audio_signals[0]
unsynced_combined_audio_crop2 = dom_output_audio_signals[2] + ben_output_audio_signals[2]

wavfile.write('/Users/dominic/Software/SDR_satellite_tracking/data/test_6_combined_unsynced_crop0.wav',
              int(audio_signal_fs), unsynced_combined_audio_crop0)

wavfile.write('/Users/dominic/Software/SDR_satellite_tracking/data/test_6_combined_unsynced_crop2.wav',
              int(audio_signal_fs), unsynced_combined_audio_crop2)

In [ ]:
cross_correlation0 = correlate(dom_output_audio_signals[0], ben_output_audio_signals[0], mode="full")
cross_correlation2 = correlate(dom_output_audio_signals[2], ben_output_audio_signals[2], mode="full")

In [ ]:
pylab.plot(cross_correlation0)

In [ ]:
pylab.plot(cross_correlation2)

In [ ]:
signal0_lag = compute_time_lag_in_samples(dom_output_audio_signals[0], ben_output_audio_signals[0])

dom_output_audio_signal_shifted0, ben_output_audio_signal_shifted0 = shift_signals_with_lag(
    dom_output_audio_signals[0], ben_output_audio_signals[0], signal0_lag)


signal2_lag = compute_time_lag_in_samples(dom_output_audio_signals[2], ben_output_audio_signals[2])

dom_output_audio_signal_shifted2, ben_output_audio_signal_shifted2 = shift_signals_with_lag(
    dom_output_audio_signals[2], ben_output_audio_signals[2], signal2_lag)


In [ ]:
signal0_lag / audio_signal_fs, signal2_lag / audio_signal_fs

In [ ]:
synced_combined_audio_crop0 = dom_output_audio_signal_shifted0 + ben_output_audio_signal_shifted0
wavfile.write('/Users/dominic/Software/SDR_satellite_tracking/data/test_6_combined_synced_crop0.wav',
              int(audio_signal_fs), synced_combined_audio_crop0)


synced_combined_audio_crop2 = dom_output_audio_signal_shifted2 + ben_output_audio_signal_shifted2
wavfile.write('/Users/dominic/Software/SDR_satellite_tracking/data/test_6_combined_synced_crop2.wav',
              int(audio_signal_fs), synced_combined_audio_crop2)
